In [1]:
in_file =r'../../data/clean/articles.csv' 

import pandas as pd
pd.set_option('display.max_columns', 500)

import numpy as np

In [2]:
articles=pd.read_csv(in_file,index_col=False)
articles.CR.fillna('',inplace=True)
df_ref=articles.CR
refs_dict={}
i=0
for ref in df_ref:
    refs_dict[i]=[ref for ref in ref.split('; ')]
    i=i+1

In [3]:
cite = open('cite_all.csv', 'w')
cite.write('citing,cited\n')

for row in refs_dict:
    for ref in refs_dict[row]:
        cite.write(str(row)+','+ ref.replace(',','')+'\n')
cite.close()

In [4]:
cite_all=pd.read_csv('cite_all.csv')
cite_all.fillna('',inplace=True)

In [6]:
dg=cite_all.groupby('cited').count()
dg1=dg[dg.citing==1]
dg=dg[dg.citing>1]
dg.reset_index(inplace=True)

In [8]:
cite_all['cited_id']=-1
i=0
dg_cited=dg.cited.values
for x in cite_all.cited:
    try:
        ind=dg[dg.cited==x].index[0]
        cite_all.set_value(i,'cited_id',ind)
    except:
        pass
    i=i+1
    if i % 1000 ==0:
        print i,
print

1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 24000 25000 26000 27000 28000 29000 30000 31000 32000 33000 34000 35000 36000 37000 38000 39000 40000 41000 42000 43000 44000 45000 46000 47000 48000 49000 50000 51000 52000 53000 54000 55000 56000 57000 58000 59000 60000 61000 62000 63000 64000 65000 66000 67000 68000 69000 70000 71000 72000 73000 74000 75000 76000 77000 78000 79000 80000 81000 82000 83000 84000 85000 86000 87000 88000 89000


In [11]:
cite_all=cite_all[cite_all.cited_id>-1]

In [12]:
df=cite_all[['citing','cited_id']]
df.to_csv('cite_all_id.csv')

In [13]:
x=df.citing.max()
y=df.cited_id.max()
mm=np.zeros((x+1,y+1))

In [ ]:
for row in df.iterrows():
    mm[row[1][0],row[1][1]]=1

In [ ]:
bib=np.tril(np.dot(mm,mm.transpose()),-1)
bib

In [ ]:
biblio = open('bib.csv', 'w')
biblio.write('source,target,weight,type\n')

for row in range(bib.shape[0]):
    for col in range(row):
        if bib[row][col] > 0:
            biblio.write(str(row)+','+str(col)+','+str(bib[row][col])+',undirected\n')
biblio.close()

In [ ]:
df=pd.read_csv('bib.csv')
df[df.weight>40]